In [4]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

In [111]:
episodes_dur = pd.read_csv(r'D:\fall2020\case study\case study 1\episodes_dur.csv')

In [112]:
episodes_dur.nunique()

Unnamed: 0      39845
PID              1231
EpisodeID       39845
ContractType        2
duration          269
dtype: int64

In [113]:
episodes_dur.isnull().sum()

Unnamed: 0      0
PID             0
EpisodeID       0
ContractType    0
duration        0
dtype: int64

In [114]:
episodes_dur.head()

,Unnamed: 0,PID,EpisodeID,ContractType,duration
0,0,530,1,CT1,60.0
1,1,1107,3,CT2,6.0
2,2,1107,4,CT2,1.0
3,3,842,6,CT2,3.0
4,4,1178,7,CT2,16.0


In [115]:
episodes_dur['num_mins'] = ""

In [116]:
for index, row in episodes_dur.iterrows():
    if row.ContractType == 'CT1':
        if row.duration <= 3:
            episodes_dur.loc[index, 'num_mins'] = 43
        elif row.duration > 3:
            Num_weeks = np.ceil(row.duration/7)
            episodes_dur.loc[index, 'num_mins'] = 31 + (Num_weeks * 34) + 12
    elif row.ContractType == 'CT2':
        if row.duration <= 3:
            episodes_dur.loc[index, 'num_mins'] = 117
        elif row.duration > 3:
            Num_weeks = np.ceil(row.duration/7)
            episodes_dur.loc[index, 'num_mins'] = 83 + (Num_weeks * 66) + 34

In [117]:
episodes_dur.head()

,Unnamed: 0,PID,EpisodeID,ContractType,duration,num_mins
0,0,530,1,CT1,60.0,349
1,1,1107,3,CT2,6.0,183
2,2,1107,4,CT2,1.0,117
3,3,842,6,CT2,3.0,117
4,4,1178,7,CT2,16.0,315


In [118]:
episodes_dur = pd.DataFrame(episodes_dur.drop('Unnamed: 0', axis = 1))

In [119]:
count_series = episodes_dur.groupby(['PID', 'ContractType']).size()

In [120]:
count_series.head()

PID  ContractType
6    CT1              1
     CT2             65
8    CT1             25
     CT2             82
9    CT1             61
dtype: int64

In [121]:
pat_num_PID_CT = count_series.to_frame(name = 'size').reset_index()

In [122]:
pat_num_PID_CT.head()

,PID,ContractType,size
0,6,CT1,1
1,6,CT2,65
2,8,CT1,25
3,8,CT2,82
4,9,CT1,61


In [123]:
pat_num_PID_CT.shape

(1913, 3)

In [124]:
duration_PID_CT = episodes_dur.groupby(['PID', 'ContractType'], as_index=False)['num_mins'].sum()

In [125]:
duration_PID_CT.head()

,PID,ContractType,num_mins
0,6,CT1,77.0
1,6,CT2,19221.0
2,8,CT1,6345.0
3,8,CT2,24114.0
4,9,CT1,11429.0


In [126]:
duration_PID_CT.shape

(1913, 3)

In [127]:
Pat_duration_sum_SNF_CT = pat_num_PID_CT.merge(duration_PID_CT, on=['PID', 'ContractType'], how='inner')

In [128]:
Pat_duration_sum_SNF_CT.head()

,PID,ContractType,size,num_mins
0,6,CT1,1,77.0
1,6,CT2,65,19221.0
2,8,CT1,25,6345.0
3,8,CT2,82,24114.0
4,9,CT1,61,11429.0


In [129]:
Pat_duration_sum_SNF_CT["average_time_spent"] = Pat_duration_sum_SNF_CT["num_mins"]/Pat_duration_sum_SNF_CT["size"]

In [130]:
Pat_duration_sum_SNF_CT.head()

,PID,ContractType,size,num_mins,average_time_spent
0,6,CT1,1,77.0,77.000000
1,6,CT2,65,19221.0,295.707692
2,8,CT1,25,6345.0,253.800000
3,8,CT2,82,24114.0,294.073171
4,9,CT1,61,11429.0,187.360656


In [131]:
Pat_duration_sum_SNF_CT[(Pat_duration_sum_SNF_CT["PID"] == 6)&(Pat_duration_sum_SNF_CT["ContractType"] == "CT1")].iloc[0]['average_time_spent']

77.0

In [132]:
Pat_duration_sum_SNF_CT.to_csv('avg_duration_per_SNF_CT.csv', index=False)

In [133]:
def calc_CCs(pat_num, CT1_prop, CT2_prop, PID, df = Pat_duration_sum_SNF_CT):
    if pat_num < 0: return print("Negative value is not acceptable. Please enter a valid number.")
    if (CT1_prop > 1 or CT1_prop < 0) or (CT2_prop > 1 or CT2_prop < 0): return print("Proportion ranges from (0,1). Please enter a valid number.")
    
    avg_CT1_time = Pat_duration_sum_SNF_CT[(Pat_duration_sum_SNF_CT["PID"] == PID)&(Pat_duration_sum_SNF_CT["ContractType"] == "CT1")].iloc[0]['average_time_spent']
    avg_CT2_time = Pat_duration_sum_SNF_CT[(Pat_duration_sum_SNF_CT["PID"] == PID)&(Pat_duration_sum_SNF_CT["ContractType"] == "CT2")].iloc[0]['average_time_spent']

    
    CCs_num = (((pat_num*CT1_prop*avg_CT1_time)/60)/52)/40 + (((pat_num*CT2_prop*avg_CT2_time)/60)/52)/40
    
    return CCs_num

In [134]:
calc_CCs(100, 0.4, 0.6, 6)

0.16684664694280077

In [107]:
calc_CCs(-1, 0.4, 0.6, 6)

Negative value is not acceptable. Please enter a valid number.


In [108]:
calc_CCs(100, -0.4, 0.6, 6)

Proportion ranges from (0,1). Please enter a valid number.


In [109]:
calc_CCs(100, 0.4, -0.6, 6)

Proportion ranges from (0,1). Please enter a valid number.
